In [ ]:
import pandas as pd
train = pd.read_csv(r'/content/train.csv')
test = pd.read_csv(r'/content/test.csv')
print("dataset loaded!")

dataset loaded!


In [ ]:
# explore the train dataset
print("shape of train dataset =",train.shape)
print("\nInfo:")
print(train.info())
print("\Statistics: ")
print(train.describe())

shape of train dataset = (8693, 14)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB
None
\Statistics: 
               Age   RoomService     FoodCourt  ShoppingMall           Spa  \
co

<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:5: SyntaxWarning: invalid escape sequence '\S'
/tmp/ipython-input-2855621713.py:5: SyntaxWarning: invalid escape sequence '\S'
  print("\Statistics: ")


In [ ]:
# print few rows
print(train.head())

  PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S  TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S  TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False  A/0/S  TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False  F/1/S  TRAPPIST-1e  16.0  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0          0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
1        109.0        9.0          25.0   549.0    44.0       Juanna Vines   
2         43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
3          0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
4        303.0       70.0         151.0   565.0     2.0  Willy Santantines   

   Transported  
0        False  
1         True  
2        False  
3        False  
4         True  


In [ ]:
# Find if missing values
print("Null values in train dataset: ")
print(train.isnull().sum())
print("\nTotal null values:", train.isnull().sum().sum())
print("\nDuplicate values: ", train.duplicated().sum())

Null values in train dataset: 
PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

Total null values: 2324

Duplicate values:  0


In [ ]:
transported_count = train['Transported'].value_counts().sort_values(ascending=False)
print(transported_count)

Transported
True     4378
False    4315
Name: count, dtype: int64


In [ ]:
# people which are transported, comes from which planets?
transported_people_planet = train.groupby('HomePlanet')['Transported'].value_counts().sort_values(ascending=False)
print(transported_people_planet.reset_index())

  HomePlanet  Transported  count
0      Earth        False   2651
1      Earth         True   1951
2     Europa         True   1404
3       Mars         True    920
4       Mars        False    839
5     Europa        False    727


In [ ]:
# split the cabin into deck / number/ side
train[['Deck','CabinNum','Side']] = train['Cabin'].str.split('/', expand=True)
test[['Deck','CabinNum','Side']] = test['Cabin'].str.split('/', expand=True)
# convert CabinNum into numeric data type
train['CabinNum'] = pd.to_numeric(train['CabinNum'], errors='coerce')
test['CabinNum'] = pd.to_numeric(test['CabinNum'], errors='coerce')

print(train[['Deck', "CabinNum", "Side"]].head())

  Deck  CabinNum Side
0    B       0.0    P
1    F       0.0    S
2    A       0.0    S
3    A       0.0    S
4    F       1.0    S


In [ ]:
# drop columns which are not required
train = train.drop(labels=['PassengerId','Cabin','Name'], axis=1)

In [ ]:
# check whether columns are frop or not
for cols in train.columns:
  print(cols)

HomePlanet
CryoSleep
Destination
Age
VIP
RoomService
FoodCourt
ShoppingMall
Spa
VRDeck
Transported
Deck
CabinNum
Side


In [ ]:
# create labels and features
x = train.drop(labels='Transported', axis=1)
y = train['Transported']
print("labels length: ", len(y))
print("features length: ", len(x))

labels length:  8693
features length:  8693


In [ ]:
# import scikit learn modules
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# categorical columns in x
cat_cols = []
# numerical columns in y
num_cols = []
for cols in x.columns:
  if x[cols].dtypes == 'object':
    cat_cols.append(cols)
  else:
    num_cols.append(cols)

print("categorical cols:\n", cat_cols)
print("\nNumerical cols:\n", num_cols)

categorical cols:
 ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']

Numerical cols:
 ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'CabinNum']


In [ ]:
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)

(6954, 13)
(1739, 13)


In [ ]:
# create pipeline and preprocessing
cat_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]
)

num_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols),
        ('cat', cat_transformer, cat_cols)
    ]
)

In [ ]:
# create pipeline for each model
rf_pipe = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('rf', RandomForestClassifier(random_state=42))
    ]
)

lgbm_pipe = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('lgbm', LGBMClassifier(bossting_type='gbdt', random_state=42))
    ]
)

xgb_pipe = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('xgb', XGBClassifier(random_state=42))
    ]
)

In [ ]:
rf_param_grid = {
    "rf__n_estimators": [100, 300, 500],
    "rf__max_depth": [5, 10, 20],
    "rf__min_samples_split": [2, 5, 10]
}
rf_grid = GridSearchCV(
    estimator=rf_pipe,
    param_grid=rf_param_grid,
    cv=3,
    scoring="accuracy",
    n_jobs=-1
)
print("fitting model...")

# Convert CryoSleep and VIP columns to string type to handle mixed types (bool and str)
# before OneHotEncoder processes them, resolving the TypeError.
for col in ['CryoSleep', 'VIP']:
    X_train[col] = X_train[col].astype(str)
    X_test[col] = X_test[col].astype(str)

rf_grid.fit(X_train, y_train)

fitting model...


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Age',
                                                                          'RoomService',
                                                                          'FoodCourt',
                                                                          'ShoppingMall',
                                                                          'Spa',
                                                                          'VRDeck',
                                                                          'CabinNum']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value='missing',
                                                                                                        strategy='constant')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['HomePlanet',
                                                                          'CryoSleep',
                                                                          'Destination',
                                                                          'VIP',
                                                                          'Deck',
                                                                          'Side'])])),
                                       ('rf',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'rf__max_depth': [5, 10, 20],
                         'rf__min_samples_split': [2, 5, 10],
                         'rf__n_estimators': [100, 300, 500]},
             scoring='accuracy')

In [ ]:
# function to calculate the accuracy
def accuracy(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy


In [ ]:
# function to save model
import joblib
def save_model(model, file_name):
  try:
    joblib.dump(model, filename=file_name)
    print(f"{model} saved successfully!")
  except Exception as e:
    print(f"Error saving model: {e}")

In [ ]:
# function to load the model
def load_model(file_path):
  try:
    model = joblib.load(filename=file_path)
    print(f"{model} loaded successfully!")
    return model
  except Exception as e:
    print(f"Error! {e}")

In [ ]:
# convert the boolean columns into string
for col in ['CryoSleep', 'VIP']:
    X_train[col] = X_train[col].astype(str)
    X_test[col] = X_test[col].astype(str)

In [ ]:
# fit the lightgbm model using randomizedsearchcv
lgbm_param_dist = {
    "lgbm__n_estimators": [200, 300, 500],
    "lgbm__learning_rate": [0.01, 0.05, 0.1],
    "lgbm__max_depth": [-1, 5, 10],
    "lgbm__num_leaves": [31, 50, 80],
}

lgbm_random = RandomizedSearchCV(
    estimator=lgbm_pipe,
    param_distributions=lgbm_param_dist,
    n_iter=10,
    cv=3,
    scoring="accuracy",
    random_state=42,
    n_jobs=-1
)

# fit the model
lgbm_random.fit(X_train, y_train)

[LightGBM] [Warning] Unknown parameter: bossting_type
[LightGBM] [Warning] Unknown parameter: bossting_type
[LightGBM] [Info] Number of positive: 3500, number of negative: 3454
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000547 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1659
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503307 -> initscore=0.013230
[LightGBM] [Info] Start training from score 0.013230


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               ['Age',
                                                                                'RoomService',
                                                                                'FoodCourt',
                                                                                'ShoppingMall',
                                                                                'Spa',
                                                                                'VRDeck',
                                                                                'CabinNum']),
                                                                              ('cat',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(fill_value='missing',
                                                                                                              strategy=...
                                                                                                OneHotEncoder(handle_unknown='ignore'))]),
                                                                               ['HomePlanet',
                                                                                'CryoSleep',
                                                                                'Destination',
                                                                                'VIP',
                                                                                'Deck',
                                                                                'Side'])])),
                                             ('lgbm',
                                              LGBMClassifier(bossting_type='gbdt',
                                                             random_state=42))]),
                   n_jobs=-1,
                   param_distributions={'lgbm__learning_rate': [0.01, 0.05,
                                                                0.1],
                                        'lgbm__max_depth': [-1, 5, 10],
                                        'lgbm__n_estimators': [200, 300, 500],
                                        'lgbm__num_leaves': [31, 50, 80]},
                   random_state=42, scoring='accuracy')

In [ ]:
# fit the xgb model using randomizedsearchcv
xgb_param_dist = {
    "xgb__n_estimators": [200, 300, 500],
    "xgb__max_depth": [3, 5, 8],
    "xgb__learning_rate": [0.01, 0.05, 0.1],
    "xgb__subsample": [0.7, 0.8, 1.0],
    "xgb__colsample_bytree": [0.7, 0.8, 1.0]
}

xgb_random = GridSearchCV(
    estimator=xgb_pipe,
    param_grid=xgb_param_dist,
    cv=3,
    scoring="accuracy",
    n_jobs=-1
)

print("fitting model...")
xgb_random.fit(X_train, y_train)

fitting model...


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Age',
                                                                          'RoomService',
                                                                          'FoodCourt',
                                                                          'ShoppingMall',
                                                                          'Spa',
                                                                          'VRDeck',
                                                                          'CabinNum']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value='missing',
                                                                                                        strategy='const...
                                                      max_leaves=None,
                                                      min_child_weight=None,
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      multi_strategy=None,
                                                      n_estimators=None,
                                                      n_jobs=None,
                                                      num_parallel_tree=None, ...))]),
             n_jobs=-1,
             param_grid={'xgb__colsample_bytree': [0.7, 0.8, 1.0],
                         'xgb__learning_rate': [0.01, 0.05, 0.1],
                         'xgb__max_depth': [3, 5, 8],
                         'xgb__n_estimators': [200, 300, 500],
                         'xgb__subsample': [0.7, 0.8, 1.0]},
             scoring='accuracy')

In [ ]:
# print best parameters
print("best parameter of the XGB model: ", xgb_random.best_params_)

best parameter of the XGB model:  {'xgb__colsample_bytree': 1.0, 'xgb__learning_rate': 0.1, 'xgb__max_depth': 3, 'xgb__n_estimators': 300, 'xgb__subsample': 1.0}


In [ ]:
# calculate the accuracy
xgb_acc = accuracy(model=xgb_random.best_estimator_, X_test=X_test, y_test=y_test)
print("accuracy of XGB model: ", xgb_acc)

accuracy of XGB model:  0.8050603795284647


In [ ]:
# print the best estimator and parameter
print("Best estimator:", lgbm_random.best_estimator_)
print("Best parameters: ", lgbm_random.best_params_)

Best estimator: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'RoomService',
                                                   'FoodCourt', 'ShoppingMall',
                                                   'Spa', 'VRDeck',
                                                   'CabinNum']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                         

In [ ]:
# caculate accuracy of best estimator for lgbm model
lgbm_accuracy = accuracy(model=lgbm_random.best_estimator_, X_test=X_test, y_test=y_test)
print("Accuracy of lgbm model:", lgbm_accuracy)

[LightGBM] [Warning] Unknown parameter: bossting_type
Accuracy of lgbm model: 0.8056354226566993


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
# save the model
save_model(model=lgbm_random.best_estimator_, file_name='lgbm_model.pkl')

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'RoomService',
                                                   'FoodCourt', 'ShoppingMall',
                                                   'Spa', 'VRDeck',
                                                   'CabinNum']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                         

In [ ]:
# function to create submission file
def create_submission(model, test_data, file_name):
  # store Passangerid
  test_id = test_data['PassengerId']
  # drop few columns which were not used  while training model
  test_data = test_data.drop(labels=['PassengerId','Cabin','Name'], axis=1)
  # predict the values on test datataset
  try:
    y_pred = model.predict(test_data)
  except Exception as e:
    print(f"Error while predicting values! {e}")
  # create a dataframe
  submission = pd.DataFrame(
      {
          'PassengerId': test_id,
          'Transported': y_pred
      }
  )
  # save dataframe in CSV
  try:
    submission.to_csv(file_name, index=False)
    print(f"{file_name} saved !")
  except Exception as e:
    print(f"error while saving file: {e}")

In [ ]:
# create submission for rf model
create_submission(model=rf_grid, test_data=test, file_name='submission_1.csv')

submission_1.csv saved !


In [ ]:
# create submission file for the lgbm model
create_submission(model=lgbm_random.best_estimator_, test_data=test, file_name='submission_2.csv')

[LightGBM] [Warning] Unknown parameter: bossting_type
submission_2.csv saved !


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
# save rf model
save_model(model=rf_grid, file_name='rf_model.pkl')

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Age',
                                                                          'RoomService',
                                                                          'FoodCourt',
                                                                          'ShoppingMall',
                                                             

In [ ]:
# save the xgb model
save_model(model=xgb_random.best_estimator_, file_name='xgb_model.pkl')

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'RoomService',
                                                   'FoodCourt', 'ShoppingMall',
                                                   'Spa', 'VRDeck',
                                                   'CabinNum']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                         

In [ ]:
# create submission file using the xgb
create_submission(model=xgb_random.best_estimator_, test_data=test, file_name='submission_3.csv')

submission_3.csv saved !
